In [ ]:
import numpy as np 
from scipy.io import loadmat
import sys 
import os

def set_path():
    if sys.platform == 'darwin':
        print("Current system is macOS")
        main_fold_path = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction'
    elif sys.platform == 'linux':
        print("Current system is Ubuntu")
        main_fold_path = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning'
    else:
        print("Current system is neither macOS nor Ubuntu")
    os.chdir(main_fold_path)
    sys.path.append(main_fold_path)    

set_path()
from utils.utils_mine import normalize_individual

In [ ]:
feature_pth = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/allData/diagnosis514/extracted_featues.npy'
label_pth = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/allData/diagnosis514/label.npy'

feature = np.load(feature_pth)
label = np.load(label_pth)


def normalize_along_features(data):
    """ 
    Args: 
        data: numpy array of shape (num_samples, num_features)
        
    Returns:
        data: numpy array of shape (num_samples, num_features)
    """

    norm_mean = np.mean(data, axis=0)
    norm_std = np.std(data, axis=0)
    norm_data = (data - norm_mean) / norm_std
    return norm_data

normalized_feature = normalize_along_features(feature)

HC = normalized_feature[label == 0]
MDD = normalized_feature[label == 1]
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import fdrcorrection


# Perform t-tests for each feature
t_stat, p_values = ttest_ind(HC, MDD, axis=0)

# Apply FDR correction
rejected, p_values_corrected = fdrcorrection(p_values)

# Define significance level
significance_level = 0.0005

# Find significant features
significant_features = np.where((rejected) & (p_values_corrected < significance_level))[0]

print("Significant features after FDR correction:")
print(len(significant_features), significant_features)
print("Corrected p-values for significant features:")
print(p_values_corrected[significant_features])

np.save('/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/allData/diagnosis514/sig_normalized_feature.npy', normalized_feature[:, significant_features])